# Regression avec un réseau de neurone simple pytorch

In [3]:
import numpy as np
import plotnine as pn
import torch
from sklearn import preprocessing
from torchinfo import summary

from adl.sklearn import skl_regression

pn.theme_set(pn.theme_minimal())

Dans les tp précédents, on a utiliser la déscente de gradient pour résoudre de simple problème de régression linéaire. Ici nous allons introduire un réseau de neurone simple pour résoudre le même problème. On va également repartir des mêmes données.

In [92]:
temperature = [-1.5, 0.2, 3.4, 4.1, 7.8, 13.4, 18.0, 21.5, 32.0, 33.5]
icecream = [100.5, 110.2, 133.5, 141.2, 172.8, 225.1, 251.0, 278.9, 366.7, 369.9]

Comme nous l'avons vu dans le tp sur la normalisation des données, on va normaliser la temperature pour améliorer l'entraînement. Et on va calculer les vrai solution , pente et ordonnée à l'origine avec scikit-learn.

In [93]:
temperature_s = preprocessing.scale(temperature, with_mean=True)

reg = skl_regression(temperature_s, icecream)
print(f"slope: {reg['slope']:.2f}, intercept: {reg['intercept']:.2f}, mse: {reg['mse']:.4f}")


slope: 95.03, intercept: 214.98, mse: 14.5403


Enfin, nous transformons nos données et nos valeur cibles en tensors. La différence ici, est que nous avons besoins d'organiser nos données de manière à avoir chaque oberservation et chaque valeur cible dans son propre tableau.

In [94]:
x = torch.tensor(temperature_s).float().view(-1, 1)
y = torch.tensor(icecream).float().view(-1, 1)

print(x)
print(y)

tensor([[-1.2238],
        [-1.0827],
        [-0.8170],
        [-0.7589],
        [-0.4517],
        [ 0.0133],
        [ 0.3952],
        [ 0.6858],
        [ 1.5576],
        [ 1.6822]])
tensor([[100.5000],
        [110.2000],
        [133.5000],
        [141.2000],
        [172.8000],
        [225.1000],
        [251.0000],
        [278.9000],
        [366.7000],
        [369.9000]])


# Régression avec pytorch et un réseau neuronal à neurone unique

Dans les notebooks précédents, nous avons créé notre modèle en créant simplement une
fonction `forward` simple, comme ceci :

``` python
def forward(x):
    return w * x + b
```

Cette approche convient pour un modèle simple comme celui que nous avons actuellement, mais pour des
modèles plus complexes tels que les réseaux neuronaux, nous devrons utiliser les
fonctions intégrées de pytorch pour les définir.

En fait, une régression linéaire simple avec une seule variable explicative
peut être considérée comme un « réseau » neuronal avec un seul neurone. Nous allons donc
essayer de réécrire notre modèle simple en utilisant la notation de pytorch.

Une façon de définir notre « réseau » consiste à utiliser la notation *Module*,
fournie par `torch.nn.Module`. Cette notation oblige à créer une nouvelle
classe Python, qui hérite de `nn.Module`, puis à créer au
moins une méthode `__init__()` (appelée lors de la création du modèle) et une
méthode `forward()`, qui prend les données d'entrée comme argument, applique notre
modèle et renvoie les valeurs prédites.

Pour créer notre modèle de régression linéaire simple, nous utiliserons `nn.Linear`,
qui permet de définir des couches linéaires de taille arbitraire. Ici, notre couche
aura un seul neurone qui prendra un seul nombre en entrée (une
valeur de température) et produira un seul résultat (un volume de vente de crème glacée prédit).
 En notation pytorch, cela signifie que notre couche aura
`in_features` de taille 1 et `out_features` de taille 1.

Voici le code d'une classe `LinearNetwork` qui implémente ce modèle.

In [9]:
from torch import nn


class LinearNetwork(nn.Module):
    """
    Modèle de régression linéaire simple avec une seule variable d'entrée.
    """

    def __init__(self):
        # Call the parent constructor (mandatory)
        super().__init__()
        # Create a "linear" attribute which will contain a linear layer with input and
        # output of size 1
        self.linear = nn.Linear(in_features=1, out_features=1)

    def forward(self, x):
        """
        Méthode qui implémente la passe avant du modèle, c'est-à-dire qui prend les données
        d'entrée en argument, applique le modèle et renvoie le résultat.
        """
        # Apply our linear layer to input data
        return self.linear(x)


Une fois que notre classe modèle est créer, on peut l'utiliser pour créer une instance de modèle.

In [11]:
model = LinearNetwork()
summary(model)

Layer (type:depth-idx)                   Param #
LinearNetwork                            --
├─Linear: 1-1                            2
Total params: 2
Trainable params: 2
Non-trainable params: 0

Il est important de distinguer entre :

-   une classe de modèle, telle que `LinearNetwork`, qui est une classe Python
    décrivant une architecture de modèle
-   un objet modèle ou une instance de modèle, tel que `model`, qui est un modèle concret
    créé à l'aide de l'architecture `LinearNetwork`

Nous pouvons utiliser la fonction `summary` du package `torchinfo` pour afficher
une description de notre objet `model`.

Nous pouvons voir que « model » comporte une couche et deux paramètres : le poids et
le biais de notre « neurone » unique. Nous pouvons voir que PyTorch se charge de
créer ces paramètres, nous n'avons plus besoin de créer manuellement les tenseurs « w » et « b »
.

Nous pouvons transmettre les données d'entrée directement à notre objet « model ». Dans ce cas, il appellera « model.forward() », qui applique le modèle aux données d'entrée
pour calculer la prédiction. Nous pouvons voir que les deux sont équivalents (les
prédictions ici sont aléatoires car les paramètres « model » ont été
initialisés de manière aléatoire lors de la création de « model »).

In [16]:
model(x)
model.forward(x)  


tensor([[ 0.9135],
        [ 0.8532],
        [ 0.7398],
        [ 0.7149],
        [ 0.5838],
        [ 0.3852],
        [ 0.2221],
        [ 0.0981],
        [-0.2742],
        [-0.3274]], grad_fn=<AddmmBackward0>)

Nous pouvons maintenant construire notre processus d'entraînement. Nous utiliserons `MSELoss()` comme fonction de perte
et un optimiseur `SGD` avec un taux d'apprentissage de 0,1. Cependant,
au lieu de passer explicitement une liste de paramètres comme `[w, b]` comme premier
argument de l'optimiseur, nous utiliserons `model.parameters()` qui
fournira automatiquement tous les paramètres de notre objet `model`.

In [17]:
loss_fn = nn.MSELoss()
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Enfin, nous définissons et exécutons notre boucle d'entraînement pendant un certain nombre d'
époches :

-   nous commençons par réinitialiser notre gradient avec `optimizer.zero_grad()`
-   nous calculons les valeurs prédites en appliquant notre objet `model` aux
    données d'entrée (passage direct)
- nous calculons la valeur de perte
- nous calculons le gradient de perte pour chaque paramètre (rétropropagation)
- enfin, nous ajustons nos paramètres en appelant `optimizer.step()`

In [21]:
epochs = 1000
for epoch in range(epochs):
    # Mettre le modèle en mode entraînement - important pour la normalisation par lots et les
    # couches de dropout. Inutile dans cette situation mais ajouté pour les meilleures pratiques
    model.train()
    # Réinitialiser les gradients des paramètres à chaque itération
    optimizer.zero_grad()
    # Forward pass: calculer les prédictions
    y_pred = model(x)
    # Calculer la loss
    loss = loss_fn(y_pred, y)
    # Backpropagation
    loss.backward()
    # Ajuster les paramètres
    optimizer.step()

    print(
        f"{epoch + 1:2}. loss: {loss:7.1f}, weight: {model.linear.weight.item():5.2f},"
        f" bias: {model.linear.bias.item():6.2f}"
    )

#slope: 95.03, intercept: 214.98, mse: 14.5403

 1. loss:   207.3, weight: 89.50, bias: 202.55
 2. loss:   199.6, weight: 89.61, bias: 202.80
 3. loss:   192.3, weight: 89.72, bias: 203.04
 4. loss:   185.3, weight: 89.82, bias: 203.28
 5. loss:   178.5, weight: 89.93, bias: 203.51
 6. loss:   172.0, weight: 90.03, bias: 203.74
 7. loss:   165.8, weight: 90.13, bias: 203.97
 8. loss:   159.8, weight: 90.23, bias: 204.19
 9. loss:   154.0, weight: 90.32, bias: 204.40
10. loss:   148.5, weight: 90.42, bias: 204.62
11. loss:   143.2, weight: 90.51, bias: 204.82
12. loss:   138.1, weight: 90.60, bias: 205.03
13. loss:   133.2, weight: 90.69, bias: 205.23
14. loss:   128.5, weight: 90.78, bias: 205.42
15. loss:   124.0, weight: 90.86, bias: 205.61
16. loss:   119.7, weight: 90.95, bias: 205.80
17. loss:   115.5, weight: 91.03, bias: 205.98
18. loss:   111.5, weight: 91.11, bias: 206.16
19. loss:   107.7, weight: 91.19, bias: 206.34
20. loss:   104.0, weight: 91.26, bias: 206.51
21. loss:   100.4, weight: 91.34, bias: 206.68
22. loss:    

On peut voir, avec beaucoup d'epoc, que le modèle atteint les vrai valeurs calculer précédemment.

# Regression avec deux variables explicatives

Si nous voulons effectuer une régression linéaire avec deux variables explicatives, nos
données d'entrée `X` seront désormais un tableau à deux colonnes.

In [95]:
# Input data
temperature = [-1.5, 0.2, 3.4, 4.1, 7.8, 13.4, 18.0, 21.5, 32.0, 33.5]
humidity = [50.1, 34.8, 51.3, 64.1, 47.8, 53.4, 58.0, 71.5, 32.0, 43.5]
icecream = [100.5, 110.2, 133.5, 141.2, 172.8, 225.1, 251.0, 278.9, 366.7, 369.9]


X = np.array([temperature, humidity]).transpose()
X


array([[-1.5, 50.1],
       [ 0.2, 34.8],
       [ 3.4, 51.3],
       [ 4.1, 64.1],
       [ 7.8, 47.8],
       [13.4, 53.4],
       [18. , 58. ],
       [21.5, 71.5],
       [32. , 32. ],
       [33.5, 43.5]])

In [96]:
X = preprocessing.scale(X)
X = torch.tensor(X).float()
y = torch.tensor(icecream).float().view(-1, 1) # .view permet de reshaper le tenseur en une colonne
print(X)
print(y)

tensor([[-1.2238, -0.0476],
        [-1.0827, -1.3712],
        [-0.8170,  0.0562],
        [-0.7589,  1.1635],
        [-0.4517, -0.2466],
        [ 0.0133,  0.2379],
        [ 0.3952,  0.6358],
        [ 0.6858,  1.8037],
        [ 1.5576, -1.6134],
        [ 1.6822, -0.6185]])
tensor([[100.5000],
        [110.2000],
        [133.5000],
        [141.2000],
        [172.8000],
        [225.1000],
        [251.0000],
        [278.9000],
        [366.7000],
        [369.9000]])


On peut maintenant normaliser nos données et les convertir en tensor comme d'habitude.

**Exercice 1**

-   Créez une nouvelle classe de modèle `LinearNetwork2` qui aura la même
    architecture que `LinearNetwork` mais qui acceptera des données d'entrée de
    dimension 2.
-   Créez un nouvel objet `model2` à partir de la classe `LinearNetwork2`
-   Affichez une description sommaire de `model2`
-   Exécutez une boucle d'apprentissage de `model2` pendant 20 époques avec une perte `MSELoss`
    et un optimiseur `SGD` avec un taux d'apprentissage de 0,1

*Astuce :* si vous souhaitez afficher les valeurs des poids et des biais à chaque
époque, vous pouvez utiliser `model2.linear.weight.data` et
`model2.linear.bias.item()`.

In [83]:
from torch import nn 

class LinearNetwork2(nn.Module):
    
    def __init__(self):
    
        super().__init__()

        self.linear = nn.Linear(in_features=2, out_features=1)
    
    def forward(self,x):
        return self.linear(x)


model2 = LinearNetwork2()
summary(model2)

Layer (type:depth-idx)                   Param #
LinearNetwork2                           --
├─Linear: 1-1                            3
Total params: 3
Trainable params: 3
Non-trainable params: 0

In [ ]:
model2.forward(X)

loss_fn2 = nn.MSELoss() # fonction de perte distance entre y_pred et y 
lr2 = 0.1 
optimizer2=torch.optim.SGD(model2.parameters(),lr=lr2)

epochs = 20

for epoch in range(epochs):
    model2.train()
    optimizer2.zero_grad()
    y_pred=model2(X)
    loss = loss_fn2(y_pred,y)
    loss.backward()
    optimizer2.step()

    print(f"{epoch + 1:2}. loss: {loss:7.1f}, weight: {model2.linear.weight.data},"
          f" bias: {model2.linear.bias.item():6.2f}"
    )



    

 1. loss: 55166.4, weight: tensor([[18.4822, -2.7803]]), bias:  43.39
 2. loss: 35264.2, weight: tensor([[33.7173, -4.3484]]), bias:  77.71
 3. loss: 22560.5, weight: tensor([[45.8634, -5.1955]]), bias: 105.17
 4. loss: 14445.7, weight: tensor([[55.5576, -5.5485]]), bias: 127.13
 5. loss:  9258.4, weight: tensor([[63.3036, -5.5716]]), bias: 144.70
 6. loss:  5940.0, weight: tensor([[69.4998, -5.3830]]), bias: 158.75
 7. loss:  3815.6, weight: tensor([[74.4617, -5.0665]]), bias: 170.00
 8. loss:  2454.6, weight: tensor([[78.4398, -4.6806]]), bias: 179.00
 9. loss:  1582.1, weight: tensor([[81.6325, -4.2656]]), bias: 186.19
10. loss:  1022.3, weight: tensor([[84.1978, -3.8481]]), bias: 191.95
11. loss:   662.9, weight: tensor([[86.2612, -3.4456]]), bias: 196.56
12. loss:   432.0, weight: tensor([[87.9227, -3.0684]]), bias: 200.24
13. loss:   283.5, weight: tensor([[89.2619, -2.7222]]), bias: 203.19
14. loss:   187.9, weight: tensor([[90.3426, -2.4095]]), bias: 205.55
15. loss:   126.4, w

# Interprétation 

Pour la lecture des résultats, ici on peut prendre directement les poids et donner : 

- $ventes_glaces = 93 × (température_norm) - 1.16 × (humidité_norm) + 212.5$

- On peut soit donner une interprétation direct disant que la température est le facteur déterminer pour la vente de glace ou alors si on veut des coefficients interprétables, il faut recalculer les coefficients pour les données originales.

# Généralisation à un nombre quelconque de variables explicatives

**Exercice 2**

Nous avons créé deux classes différentes ci-dessus : une pour un modèle de régression linéaire
avec une seule variable explicative, et une pour deux variables explicatives.
 Nous allons maintenant essayer de créer une classe de modèle plus générique qui peut
renvoyer des modèles acceptant un nombre quelconque de variables explicatives.

- Créez une nouvelle classe `GeneralLinearNetwork` en partant de la
  classe `LinearNetwork` vue ci-dessus
- Modifiez la méthode `__init__()` afin qu'elle accepte un nouvel argument
  appelé `n_variables`
- Modifiez la création de `self.linear` afin qu'elle prenne en compte la
  valeur passée en tant qu'argument `n_variables`

Une fois la classe créée :

- Instanciez un objet modèle appelé `model1` qui accepte des données d'entrée
  avec une colonne et appliquez-le aux données d'entrée `x`.
- Instanciez un objet modèle appelé `model2` qui accepte des données d'entrée
  avec deux colonnes et appliquez-le aux données d'entrée `X`.

In [106]:
def train_model(data,model,lr,optimizer,loss_fn,epochs):
    
    for epoch in range(epochs):
      model.train()
      optimizer.zero_grad()
      y_pred=model(data)
      loss = loss_fn(y_pred,y)
      loss.backward()
      optimizer.step()

      print(f"{epoch + 1:2}. loss: {loss:7.1f}, weight: {model.linear.weight.data},"
          f" bias: {model.linear.bias.item():6.2f}"
    )

In [133]:



class GeneralLinearNetwork(nn.Module):
    
    def __init__(self,n_variables):
        super().__init__()
        self.linear = nn.Linear(in_features=n_variables,out_features=1)

    def forward(self,x):
        return self.linear(x)
        
model1 = GeneralLinearNetwork(1)
model1(x) # model1.forward(x) : identique
loss_fn = nn.MSELoss() # fonction de perte distance entre y_pred et y 
lr = 0.1 

epochs = 20
optimizer1=torch.optim.SGD(model1.parameters(),lr=lr)

train_model(x,model1,lr,optimizer1,loss_fn,epochs)


 1. loss: 55526.9, weight: tensor([[19.4812]]), bias:  42.29
 2. loss: 35542.4, weight: tensor([[34.5908]]), bias:  76.83
 3. loss: 22752.4, weight: tensor([[46.6785]]), bias: 104.46
 4. loss: 14566.8, weight: tensor([[56.3486]]), bias: 126.56
 5. loss:  9328.0, weight: tensor([[64.0848]]), bias: 144.25
 6. loss:  5975.1, weight: tensor([[70.2737]]), bias: 158.39
 7. loss:  3829.3, weight: tensor([[75.2248]]), bias: 169.71
 8. loss:  2456.0, weight: tensor([[79.1857]]), bias: 178.77
 9. loss:  1577.1, weight: tensor([[82.3544]]), bias: 186.01
10. loss:  1014.6, weight: tensor([[84.8894]]), bias: 191.80
11. loss:   654.6, weight: tensor([[86.9173]]), bias: 196.44
12. loss:   424.1, weight: tensor([[88.5397]]), bias: 200.15
13. loss:   276.7, weight: tensor([[89.8376]]), bias: 203.11
14. loss:   182.3, weight: tensor([[90.8759]]), bias: 205.49
15. loss:   121.9, weight: tensor([[91.7066]]), bias: 207.39
16. loss:    83.3, weight: tensor([[92.3711]]), bias: 208.90
17. loss:    58.5, weigh

In [136]:
model2 = GeneralLinearNetwork(2)
model2(X) # model2.forward(X) : identique
loss_fn2 = nn.MSELoss() # fonction de perte distance entre y_pred et y 
lr2 = 0.1 
optimizer2=torch.optim.SGD(model2.parameters(),lr=lr2)

epochs2 = 20

train_model(X,model2,lr2,optimizer2,loss_fn2,epochs2)

 1. loss: 55134.3, weight: tensor([[18.6782, -2.7963]]), bias:  43.37
 2. loss: 35244.1, weight: tensor([[33.8736, -4.3559]]), bias:  77.69
 3. loss: 22547.9, weight: tensor([[45.9883, -5.1974]]), bias: 105.15
 4. loss: 14437.7, weight: tensor([[55.6575, -5.5466]]), bias: 127.11
 5. loss:  9253.3, weight: tensor([[63.3836, -5.5675]]), bias: 144.69
 6. loss:  5936.7, weight: tensor([[69.5638, -5.3776]]), bias: 158.75
 7. loss:  3813.5, weight: tensor([[74.5131, -5.0604]]), bias: 169.99
 8. loss:  2453.3, weight: tensor([[78.4811, -4.6744]]), bias: 178.99
 9. loss:  1581.2, weight: tensor([[81.6657, -4.2595]]), bias: 186.19
10. loss:  1021.7, weight: tensor([[84.2245, -3.8424]]), bias: 191.95
11. loss:   662.5, weight: tensor([[86.2827, -3.4403]]), bias: 196.55
12. loss:   431.7, weight: tensor([[87.9400, -3.0636]]), bias: 200.24
13. loss:   283.3, weight: tensor([[89.2760, -2.7179]]), bias: 203.19
14. loss:   187.8, weight: tensor([[90.3540, -2.4057]]), bias: 205.55
15. loss:   126.3, w